<a href="https://www.kaggle.com/code/prayagbarath/veritas-ai-random-forest-satellite-orbit-collision?scriptVersionId=290473492" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import warnings
import pickle
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df = pd.read_csv('/kaggle/input/satellite-orbital-data/train_data.csv')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/satellite-orbital-data/train_data 2.csv
/kaggle/input/satellite-orbital-data/train_data.csv


In [2]:
categorical_columns = []
for column in df.columns:
    df[column] = (pd.to_numeric(df[column], errors="coerce")).fillna(0)
    if (df[column].max() == df[column].min()==0):
        categorical_columns.append(column)

    if(df[column].max()>1e10):
        indices = np.where(df[column]>1e10)[0]
        df.loc[indices, column] = 1e10



print (categorical_columns)
    
    

['c_object_type']


In [ ]:
def SF_create_random_forest(column, num_estimators):
    X_single = np.array(df[column].fillna(0)).reshape(-1,1)
    y = np.array(df['risk'])
    bins = [-np.inf, -15, -5, 5, np.inf]
    labels = ['high', 'medium', 'low', 'very_low'] 
    
    y = pd.cut(y, bins=bins, labels=labels) #allows splitting of continuous values into variables based on provided bins 

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
    X_single, y, test_size=0.2, random_state=7
    )

    # Random Forest
    model = RandomForestClassifier(n_estimators = num_estimators, random_state=7)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print ("Risk is estimated to be", y_pred, "based on", column)
    print(f"Accuracy for miss: {accuracy:.2f}")
    return accuracy

In [3]:
#time series step 1
eids = np.unique(df["event_id"])
max_len_eid = 0
for eid in eids:
  inds = np.where(df["event_id"] == eid)[0] #which rows correspond to each event id; [0] to ensure 1D array
  max_len_eid = max(max_len_eid, len(str(len(inds)))) #len(str(len)) --> how many digits is the max num of times in an eid

print(max_len_eid)

2


In [4]:
#time series step 2
max_len_column = 0
for column in df.columns:
  max_len_column = max(max_len_column, len(column)) #num symbols per column

print(max_len_column)

25


In [6]:
#time series step 3: fxn
def time_series_cleaning():
    new_df = None
    for eid in eids:
        inds = np.where(df["event_id"] == eid)[0]
        columns = np.zeros(len(inds)*len(df.columns), dtype=f"U{max_len_column+max_len_eid}") #dtype (datatype) U means Unicode for String: Creates a String that can store x many values
        values  = np.zeros(len(inds)*len(df.columns))
    
        cnt = -1
        for i, ind in enumerate(inds): #enumerate --> pass a list and it provides index and value
          for j, column in enumerate(df.columns):
            cnt += 1
            columns[cnt] = str(column)+str(i)
            try: #attempt the code inside,  but if error encountered defer to except block
              values[cnt] = df.iloc[ind][column] #look at a row specified by ind and grab the value stored for the "column" at that row
            except:
              values[cnt] = np.nan # handle non-numeric types
        
        if new_df is None:
          new_df = pd.DataFrame(data=values.reshape(1, -1), columns=columns)
        else:
          new_df = pd.concat([new_df, pd.DataFrame(data=values.reshape(1, -1), columns=columns)]) #concatenates each unique id iteration together

    with open("/kaggle/working/train_data.pkl", "wb") as f:
        pickle.dump(new_df, f)

In [7]:
time_series_cleaning()

In [ ]:
max_len_column = 0
for column in df.columns:
  max_len_column = max(max_len_column, len(column))

print(max_len_column)

In [ ]:
#10 tree forest
accuracy_of_estimates10sf = []
for column in df.columns:
    if column in categorical_columns:
        continue
    accuracy_of_estimates10sf.append(SF_create_random_forest(column, 10))

print (accuracy_of_estimates10sf)

In [ ]:
save_dictionary = {'10 Trees':accuracy_of_estimates10sf}
df_save= pd.DataFrame(save_dictionary)
df_save.to_pickle('10TreeSingleFeatureAnalysis.pkl')

In [ ]:
#barcharts

accuracies = accuracy_of_estimates10sf 
indices = np.argsort(np.array(accuracies))[::-1]
accuracies = np.array(accuracies)[indices]
columns = np.array(df.columns)[indices]
plt.xticks(rotation=90)
print (accuracies)
plt.bar(columns[:10], accuracies[:10], width=.5)

In [ ]:
#30 trees
accuracy_of_estimates30sf = []
for column in df.columns:
    if column in categorical_columns:
        continue
    accuracy_of_estimates30sf.append(SF_create_random_forest(column, 30))

print (accuracy_of_estimates30sf)

In [ ]:
save_dictionary = {'30 Trees':accuracy_of_estimates30sf}
df_save= pd.DataFrame(save_dictionary)
df_save.to_pickle('30TreeSingleFeatureAnalysis.pkl')

In [ ]:
#barcharts

accuracies = accuracy_of_estimates30sf
indices = np.argsort(np.array(accuracies))[::-1]
accuracies = np.array(accuracies)[indices]
columns = np.array(df.columns)[indices]
plt.xticks(rotation=90)
print (accuracies)
plt.bar(columns[:10], accuracies[:10], width=.5)

In [ ]:
#100 trees
accuracy_of_estimates = []
for column in df.columns:
    if column in categorical_columns:
        continue
    accuracy_of_estimates.append(SF_create_random_forest(column, 100))

print (accuracy_of_estimates)

In [ ]:
save_dictionary = {'100 Trees':accuracy_of_estimates}
df_save= pd.DataFrame(save_dictionary)
df_save.to_pickle('100TreeSingleFeatureAnalysis.pkl')

In [ ]:
#barcharts

accuracies = accuracy_of_estimates 
indices = np.argsort(np.array(accuracies))[::-1]
accuracies = np.array(accuracies)[indices]
columns = np.array(df.columns)[indices]
plt.xticks(rotation=90)
print (accuracies)
plt.bar(columns[:10], accuracies[:10], width=.5)

In [ ]:
def DF_create_random_forest(column1, column2, num_estimators):
    features = [column1, column2]
    X = ((df[features].to_numpy()))#.reshape(-1,1)
    y = np.array(df['risk'])
    bins = [-np.inf, -15, -5, 5, np.inf]
    labels = ['high', 'medium', 'low', 'very_low'] 
    
    y = pd.cut(y, bins=bins, labels=labels) #allows splitting of continuous values into variables based on provided bins 

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=7
    )

    # Random Forest
    model = RandomForestClassifier(n_estimators=num_estimators, random_state=7)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print ("Risk is estimated to be", y_pred, "based on", column1, "and", column2)
    print(f"Accuracy for miss: {accuracy:.2f}")
    return accuracy

In [ ]:
#10 trees
accuracy_of_estimates10df = []
dual_column_list=[]
for i in range(0,len(df.columns)-1,2):
    if column in categorical_columns:
        continue
    accuracy_of_estimates10df.append(DF_create_random_forest(df.columns[i], df.columns[i+1], 10))
    dual_column_list.append([df.columns[i],df.columns[i+1]])

print (accuracy_of_estimates10df)

In [ ]:
save_dictionary = {'10 Trees':accuracy_of_estimates10df}
df_save= pd.DataFrame(save_dictionary)
df_save.to_pickle('10TreeDoubleFeatureAnalysis.pkl')

In [ ]:
#barchartsdf

accuracies = accuracy_of_estimates10df
indices = np.argsort(np.array(accuracies))[::-1]
accuracies = np.array(accuracies)[indices]
columns = np.array(dual_column_list)[indices]
columns = [" & ".join(col) if isinstance(col, (list, np.ndarray)) else str(col) 
           for col in columns]
print (columns)
plt.xticks(rotation=90)
print (accuracies)
plt.bar(columns[:10], accuracies[:10], width=.5)

In [ ]:
#30 trees
accuracy_of_estimates30df = []
dual_column_list=[]
for i in range(0,len(df.columns)-1,2):
    if column in categorical_columns:
        continue
    accuracy_of_estimates30df.append(DF_create_random_forest(df.columns[i], df.columns[i+1], 30))
    dual_column_list.append([df.columns[i],df.columns[i+1]])

print (accuracy_of_estimates30df)

In [ ]:
save_dictionary = {'30 Trees':accuracy_of_estimates30df}
df_save= pd.DataFrame(save_dictionary)
df_save.to_pickle('30TreeDoubleFeatureAnalysis.pkl')

In [ ]:
#barchartsdf

accuracies = accuracy_of_estimates30df
indices = np.argsort(np.array(accuracies))[::-1]
accuracies = np.array(accuracies)[indices]
columns = np.array(dual_column_list)[indices]
columns = [" & ".join(col) if isinstance(col, (list, np.ndarray)) else str(col) 
           for col in columns]
print (columns)
plt.xticks(rotation=90)
print (accuracies)
plt.bar(columns[:10], accuracies[:10], width=.5)

In [ ]:
#100 trees
accuracy_of_estimatesdf = []
dual_column_list=[]
for i in range(0,len(df.columns)-1,2):
    if column in categorical_columns:
        continue
    accuracy_of_estimatesdf.append(DF_create_random_forest(df.columns[i], df.columns[i+1], 100))
    dual_column_list.append([df.columns[i],df.columns[i+1]])

print (accuracy_of_estimatesdf)

In [ ]:
save_dictionary = {'100 Trees':accuracy_of_estimatesdf}
df_save= pd.DataFrame(save_dictionary)
df_save.to_pickle('100TreeDoubleFeatureAnalysis.pkl')

In [ ]:
#barchartsdf

accuracies = accuracy_of_estimatesdf
indices = np.argsort(np.array(accuracies))[::-1]
accuracies = np.array(accuracies)[indices]
columns = np.array(dual_column_list)[indices]
columns = [" & ".join(col) if isinstance(col, (list, np.ndarray)) else str(col) 
           for col in columns]
print (columns)
plt.xticks(rotation=90)
print (accuracies)
plt.bar(columns[:10], accuracies[:10], width=.5)

In [ ]:
#normalization
x = df["miss_distance"]
xmin = x.min()
x-=xmin
xmax = x.max()
x/=xmax
print (x, x.min(), x.max())

In [ ]:
#normalization
x = df["miss_distance"]
xmin = x.min()
x-=xmin
indices = np.where(x==x.min())[0]
indices2 = np.where(x!=x.min())[0]
x2 = x[indices2]
x[indices] = x2.min()
x = np.log10(x)
x-=x.min()
xmax = x.max()
x/=xmax
print (x, x.min(), x.max())